# Stock Market Simulator

## Imports

In [12]:
from datetime import datetime
import pandas as pd

<hr>

## Classes

### Timestamp

In [50]:
class Timestamp:
    def __init__ (self, date: datetime, symbol: str, open_volume: float, total_volume: float):
        self.date = date
        self.symbol = symbol
        self.open_volume = open_volume
        self.total_volume = total_volume
    
    @property
    def date (self) -> datetime:
        return self._date
    
    @date.setter
    def date (self, value: datetime):
        if not isinstance(value, datetime):
            raise TypeError("Date must be a datetime object!")
        self._date = value
    
    @property
    def symbol (self) -> str:
        return self._symbol
    
    @symbol.setter
    def symbol (self, value):
        if not isinstance(value, str):
            raise TypeError("Symbol must be string!")
        if not value:
            raise ValueError("Symbol can't be empty!")
        if not value.strip().isalpha():
            raise ValueError("Symbol must only contain alphabetical characters!")
        self._symbol = value.strip().upper()
    
    @property
    def open_volume (self) -> float:
        return self._open_volume
    
    @open_volume.setter
    def open_volume (self, value: float):
        if not isinstance(value, float):
            raise TypeError("Open volume must be float!")
        if value <= 0:
            raise ValueError("Open volume must be positive!")
        self._open_volume = value

    @property
    def total_volume (self) -> float:
        return self._total_volume
    
    @total_volume.setter
    def total_volume (self, value: float):
        if not isinstance(value, float):
            raise TypeError("Total volume must be float!")
        if value <= 0:
            raise ValueError("Total volume must be positive!")
        self._total_volume = value
    
    def __repr__ (self):
        return f"{self.date} {self.symbol} {self.open_volume} {self.total_volume}"
    
    def __str__ (self):
        return self.__repr__()

### Account

In [14]:
class Account:
    def __init__ (self, name: str, surname: str, national_number: str, date_of_birth: datetime, balance: int):
        self.name = name
        self.surname = surname
        self.national_number = national_number
        self.date_of_birth = date_of_birth
        self.balance = balance
    
    @property
    def name (self) -> str:
        return self._name
    
    @name.setter
    def name (self, value):
        if not isinstance(value, str):
            raise TypeError("Name must be string!")
        if not value:
            raise ValueError("Name can't be empty!")
        if not value.replace(" ", "").isalpha():
            raise ValueError("Name must only contain spaces and alphabetical characters!")
        self._name = value.strip().capitalize()
    
    @property
    def surname (self) -> str:
        return self._surname
    
    @surname.setter
    def surname (self, value: str):
        if not isinstance(value, str):
            raise TypeError("Surname must be string!")
        if not value:
            raise ValueError("Surname can't be empty!")
        if not value.replace(" ", "").isalpha():
            raise ValueError("Surname must only contain spaces and alphabetical characters!")
        self._surname = value.strip().capitalize()

    @property
    def national_number (self) -> str:
        return self._national_number
    
    @national_number.setter
    def national_number (self, value: str):
        if not isinstance(value, str):
            raise TypeError("National number must be string!")
        if len(value) != 10:
            raise ValueError("National number must contain exactly 10 characters!")
        if not value.isdigit():
            raise ValueError("National number must only contain digits!")
        self._national_number = value
    
    @property
    def date_of_birth (self) -> datetime:
        return self._date_of_birth
    
    @date_of_birth.setter
    def date_of_birth (self, value: datetime):
        if not isinstance(value, datetime):
            raise TypeError("Date of birth must be a datetime object!")
        self._date_of_birth = value
    
    @property
    def balance (self) -> int:
        return self._balance
    
    @balance.setter
    def balance (self, value: int):
        if not isinstance(value, int):
            raise TypeError("Balance must be integer!")
        if value <= 0:
            raise ValueError("Balance must be positive!")
        self._balance = value
    
    def buy_shares (self, timestamp: Timestamp, count):
        pass

    def sell_shares (self, timestamp: Timestamp, count):
        pass

    def show_shares (self):
        pass

<hr>

## Initialization

In [15]:
df = pd.read_csv("data/raw.csv")

In [16]:
df.head(n = 10)

,Date,Symbol,Open,Volume
0,2009-12-31,AMZN,136.250000,7599900.0
1,2010-01-04,AMZN,136.250000,7599900.0
2,2010-01-05,AMZN,133.429993,8851900.0
3,2010-01-06,AMZN,134.600006,7178800.0
4,2010-01-07,AMZN,132.009995,11030200.0
5,2010-01-08,AMZN,130.559998,9830500.0
6,2010-01-11,AMZN,132.619995,8779400.0
7,2010-01-12,AMZN,128.990005,9096300.0
8,2010-01-13,AMZN,127.900002,10723200.0
9,2010-01-14,AMZN,129.139999,9774900.0


In [17]:
df.sample(n = 10, random_state = 1)

,Date,Symbol,Open,Volume
5241,2018-06-01,FB,193.070007,17307200.0
12377,2021-12-22,GOOGL,2861.350098,1316100.0
2005,2017-12-13,AMZN,1170.000000,2616800.0
2739,2020-11-10,AMZN,3095.020020,6591000.0
3042,2022-01-25,AMZN,2844.850098,4541200.0
15479,2021-11-29,AAPL,159.369995,88748200.0
13197,2012-11-09,AAPL,19.300714,929913600.0
12542,2010-04-08,AAPL,8.587143,572989200.0
3916,2013-03-04,FB,27.760000,32400700.0
14130,2016-07-26,AAPL,24.205000,224959200.0


In [51]:
SYMBOLS = pd.unique(df["Symbol"]).tolist()

TIMESTAMPS = {
    symbol: [
        Timestamp(
            datetime.strptime(record["Date"], "%Y-%m-%d"),
            record["Symbol"],
            record["Open"],
            record["Volume"]
        )
        for _, record in df.loc[df["Symbol"] == symbol].iterrows()
    ]
    for symbol in SYMBOLS
}

<hr>

## Visualization